<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5B%EB%8D%B0%EA%B3%BC%EC%9D%B8%5D%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%20%ED%95%99%EC%8A%B5%20%EC%8B%A4%EC%8A%B5_%EC%9D%B4%EB%8B%A4%EC%98%A8%20%EC%A1%B0%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터과학과인공지능 7분반 실습
Written by Daon Lee (onlyforres@gmail.com) on 09/25/2024

# **[신경망 학습]**

- 학습: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것

# 손실함수

- 신경망 학습에서 사용하는 지표

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# folder = '데과인'
folder = ''

In [ ]:
import sys, os
sys.path.append(f'/content/drive/MyDrive/{folder}')

import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784)
print(t_train.shape) # 원-핫 인코딩 된 정답 레이블 (60000, 10)

(60000, 784)
(60000, 10)


# 신경망에서의 기울기

- 전제

    신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다. 신경망 학습은 다음과 같이 4단계로 수행한다.

- 1단계 - 미니배치

    훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표이다.

- 2단계 - 기울기 산출

    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.

- 3단계 - 매개변수 갱신

    가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

- 4단계 - 반복

    1~3단계를 반복한다.

- 데이터를 무작위로 선정하기 때문에 확률적 경사 하강법stochastic gradient descent,
SGD라고 부른다.


++ 필요 함수들 정의

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [ ]:
def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x)  # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val  # 값 복원
        it.iternext()

    return grad

In [ ]:
######### 오차역전파법 ####### : sigmoid에 대한 오차역전파법. 다음 실습 때 쓰일 수도 ㅇㅇ.
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

# 학습 알고리즘 구현해보기

In [ ]:
class TwoLayerNet:
    """
    params : 신경망의 매개변수를 보관하는 딕셔너리 변수.
    params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향.
    params['W2']은 2번째 층의 가중치, params['b2']은 2번째 층의 편향.

    grad : 기울기를 보관하는 딕셔너리 변수(numerical_gradient()의 반환값)
    grads['W1']은 1번째 층의 가중치의 기울기, grads['b1']은 1번째 층의 편향의 기울기.
    grads['W2']은 2번째 층의 가중치의 기울기, grads['b2']은 2번째 층의 편향의 기울기.
    """
    # 초기화를 수행. input은 데이터의 크기, output size는 예측해야 할 class의 개수.
    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std=0.01):
        # 매개변수 초기화 : 편향은 0으로, 가중치는 random으로 시작함. 이거 말고도 다양한 초기화 방법 존재.
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    # 예측(추론)을 수행한다. 이게 2층짜리 딥러닝 구조의 전부.
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # 손실 함수의 값을 구한다.
    # x : 입력데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    # 정확도를 구한다. 결과가 나올 때 학습이 잘 이루어졌는지 볼 수 있는 지표.
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # 가중치 매개변수의 기울기를 구한다.
    # x : 입력 데이터, t : 정답 레이블
    # 이게 loss로 나온 결과를 수치미분하여 저장할 수 있는 코드.
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

# 미니배치 학습 구현


In [ ]:
import sys, os
sys.path.append(f'/content/drive/MyDrive/{folder}')

import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784)  6만장이고, 784의 크기를 갖고 있음.
print(t_train.shape) # 원-핫 인코딩 된 정답 레이블 (60000, 10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
# 하이퍼파라미터
iters_num = 10000  # 이걸 설정해도 되고, epoch을 설정해줘도 됨.
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1  # 학습률 안 줄이고 계속 0.1 하나로 진행.

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)  # 두 개의 층을 갖는 딥러닝 구조.

for i in range(iters_num):  # 이 iteration 수만큼 반복하게 됨.
    batch_mask = np.random.choice(train_size, batch_size)  # 랜덤하게 뽑힌 100개를 사용. 근데 이거 하면 데이터가 반복될 수 있어서, 실제로 할 땐 batch를 안 겹치게 뽑는 다른 코드 사용함.
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    # 경사하강법을 통해 파라미터를 업데이트하는 단계
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:  # 몇번 돌 때마다 정확도 등의 정보를 뽑아낼 건지.
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

(60000, 784)
(60000, 10)
train acc, test acc | 0.11236666666666667, 0.1135
train acc, test acc | 0.7923666666666667, 0.7972
train acc, test acc | 0.87605, 0.8787
train acc, test acc | 0.8968166666666667, 0.8983
train acc, test acc | 0.9081166666666667, 0.9108
train acc, test acc | 0.9136666666666666, 0.9151
train acc, test acc | 0.9181, 0.9197
train acc, test acc | 0.9224333333333333, 0.9252
train acc, test acc | 0.9264666666666667, 0.9288
train acc, test acc | 0.9306, 0.9312
train acc, test acc | 0.9328666666666666, 0.9344
train acc, test acc | 0.9349833333333334, 0.9353
train acc, test acc | 0.9384, 0.9386
train acc, test acc | 0.9400833333333334, 0.94
train acc, test acc | 0.9428666666666666, 0.9409
train acc, test acc | 0.9448166666666666, 0.9441
train acc, test acc | 0.94635, 0.9444
